In [12]:
import numpy as np
from scipy.stats import beta, uniform, bernoulli, norm, multinomial

In [61]:
theta_prior_cache = {}
probabilities_len = 21
lit_listener_shp = (probabilities_len, probabilities_len)

listener_matrix = np.tril(np.full(lit_listener_shp, 1.0, dtype=np.float64), 0) 
col_sums = np.sum(listener_matrix, axis=0)
listener_matrix = listener_matrix / col_sums

speaker_matrix = np.log(listener_matrix)

def theta_prior(a, b, exclude_lower=False, exclude_upper=False):
    key_tuple = (a, b, exclude_lower, exclude_upper)
    if key_tuple not in theta_prior_cache:
      distr = beta(a,b)
      ticks = np.arange(0.0, 1.01, 0.05)
      ticks[0] = .000001
      ticks[-1] = 1 - .000001
      probs =  distr.pdf(ticks)
      #probs[np.isfinite(probs) == False] = 0
      print(probs)
      print((probs / sum(probs)))
      theta_prior_cache[key_tuple] = multinomial(n=1, p=(probs / sum(probs)))
    return theta_prior_cache[key_tuple]

/Users/sebschu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':


In [66]:
thresholds = theta_prior(2,4).rvs(10000)
thresholds = [np.where(r==1)[0][0] for r in thresholds]

In [80]:
expr1 = np.take(speaker_matrix, indices=thresholds, axis=1)
expr2 = np.take(speaker_matrix, indices=thresholds, axis=1)
expr3 = np.take(speaker_matrix, indices=[0] * 10000, axis=1)

In [81]:
expr3 = expr3 - 2.5

In [99]:
stacked = np.stack([expr1, expr2, expr3])

In [98]:
costs = np.array([1,2,3]).reshape((3,1,1))

In [84]:
stacked = stacked / np.sum(stacked, axis=0, keepdims=True)

In [101]:
stacked - costs

array([[[       -inf,        -inf,        -inf, ...,        -inf,
                -inf,        -inf],
        [       -inf,        -inf,        -inf, ...,        -inf,
                -inf,        -inf],
        [       -inf,        -inf,        -inf, ...,        -inf,
                -inf,        -inf],
        ..., 
        [-3.63905733, -3.07944154, -3.89037176, ..., -3.7080502 ,
         -3.39789527, -3.7080502 ],
        [-3.63905733, -3.07944154, -3.89037176, ..., -3.7080502 ,
         -3.39789527, -3.7080502 ],
        [-3.63905733, -3.07944154, -3.89037176, ..., -3.7080502 ,
         -3.39789527, -3.7080502 ]],

       [[       -inf,        -inf,        -inf, ...,        -inf,
                -inf,        -inf],
        [       -inf,        -inf,        -inf, ...,        -inf,
                -inf,        -inf],
        [       -inf,        -inf,        -inf, ...,        -inf,
                -inf,        -inf],
        ..., 
        [-4.63905733, -4.07944154, -4.89037176, ...,

In [86]:
stacked = np.mean(stacked, axis=2)

In [87]:
stacked

array([[ 0.        ,  0.01972884,  0.05511483,  0.10187001,  0.1526818 ,
         0.20281413,  0.25347884,  0.29864307,  0.33926108,  0.37595061,
         0.40571081,  0.43160474,  0.45209433,  0.46509113,  0.47416696,
         0.48083872,  0.48425534,  0.48594215,  0.48638953,  0.48643933,
         0.48643933],
       [ 0.        ,  0.01972884,  0.05511483,  0.10187001,  0.1526818 ,
         0.20281413,  0.25347884,  0.29864307,  0.33926108,  0.37595061,
         0.40571081,  0.43160474,  0.45209433,  0.46509113,  0.47416696,
         0.48083872,  0.48425534,  0.48594215,  0.48638953,  0.48643933,
         0.48643933],
       [ 1.        ,  0.96054233,  0.88977035,  0.79625998,  0.69463641,
         0.59437173,  0.49304231,  0.40271385,  0.32147784,  0.24809878,
         0.18857838,  0.13679052,  0.09581133,  0.06981775,  0.05166608,
         0.03832255,  0.03148933,  0.0281157 ,  0.02722095,  0.02712134,
         0.02712134]])

In [89]:
x = np.full(21, 1)

In [90]:
x = x / np.sum(x)

In [91]:
x

array([ 0.04761905,  0.04761905,  0.04761905,  0.04761905,  0.04761905,
        0.04761905,  0.04761905,  0.04761905,  0.04761905,  0.04761905,
        0.04761905,  0.04761905,  0.04761905,  0.04761905,  0.04761905,
        0.04761905,  0.04761905,  0.04761905,  0.04761905,  0.04761905,
        0.04761905])

In [93]:
np.shape(np.stack([x]* 1000, axis=1))

(21, 1000)

In [104]:
[i for i in range(7) if i != 4 and i != 3]

[0, 1, 2, 5, 6]

In [108]:
np.shape(np.sum(stacked[[0,2], :, :], axis=0))

(21, 10000)